# ELASTIC TOY NETWORK STA

This file first shows how to run an elastic Static Traffic Assignment in dyntapy. It also shows examples of how to run an elastic assignment with tolls. You will see that there are large pieces of code the same as with the inelastics toy network file.

1. At first, you import all the necessary packages and files from mostly dyntapy. 

In [ ]:
import warnings
warnings.filterwarnings('ignore') # hide warnings
import os
import numpy as np
import sys
sys.path.append("../../..")

from pickle import dump
from dyntapy.supply_data import get_toy_network, relabel_graph
from dyntapy.demand_data import add_centroids, od_graph_from_matrix
from dyntapy.visualization import show_network
from dyntapy.assignments import StaticAssignment

2. Some toy networks are already made available in the original toolbox, so in the example,two-route is used. The first steps are always the same: 

- Create or retrieve a network
- Create and add centroids to network → adds connectors automatically
- Create or load an OD matrix (with the demand) onto the centroids

  These steps can also be visualised in the notebooks by fully running the next block of code. There are now links visible (roads and connectors), green centroids and for both elements when hovering with the cursor over the segments there are also some characteristics displayed in a box like the link ID, the capacity, the free-flow speed or demand between each location in the demand plot.

In [ ]:
# 1. Retrieve network
network = get_toy_network('two-route')

# 2. Create and add centroids
centroid_x = np.array([-1, 3])
centroid_y = np.array([2, 2])
network = add_centroids(network, centroid_x, centroid_y, euclidean=True)
# also adds connectors automatically
network = relabel_graph(network) 
show_network(network, euclidean=True, notebook= True)

# 3. Create OD and load onto centroids
old_od = np.zeros(4).reshape((2, 2))
old_od[0, 1] = 400
old_graph = od_graph_from_matrix(old_od, centroid_x, centroid_y)

3. To have an elastic assignment, the inverse demand function approach is used. To calculate the A matrix, first an STA without toll needs to be run to get a price-demand relation. This can be done using the already familiar piece of code running the assignment below. The demand that is put on the network will in the first place remain static, being the original OD graph. 

   To determine A, you should have three other elements: B (elasticity matrix from literature) and a related price-demand value as P and D. These last ones can be extracted from the previous untolled assignment. The demand D is known and the impedance or P between each origin-destination pair can be calculated via the built-in function to extract the skims. Afterwards you just fill in the inverse demand function to get a value for A.

In [ ]:
# 1. STA without toll 
assignment = StaticAssignment(network, old_graph)
result = assignment.run('dial_b')  

# 2. Other elements you need: B matrix, here a random number
B = 10

# 3. calculating A based on B, P and D
A = old_od*B + result.skim

    Congratulations, you now have every element to start the loop to determine realistic demand via elasticities!

4. When you now want to run an elastic assignemt, it is essential to add an additional loop where you determine how much the demand matrix will change depending on toll values and route choice. This can be done by checking each time after an assignment whether or not the change in skim matrix will have a significant effect. To get in this loop, you will first run an assignment with a random toll and determine how much the demand data deviates. A threshold of a specific number of iterations (here 10) or a lower bound on the max deviation per OD pair (here 5) will put a stop to the iterating step over multiple STA's. 


In [ ]:
# 1. Create toll object
toll_method = 'zone'
toll_link_ids = [9,10] # Link 9 goes from the bottom to the right, and link 10 goes from the bottom to the top. 
toll_value = 3
toll_object = create_toll_object(g, toll_method, toll_link_ids, toll_value)

# 2. Run STA with toll object
assignment2 = StaticAssignment(g,od_graph, toll_object)
result2 = assignment2.run('dial_b')
print('dial_b ran successfully')
show_network(g, flows = result2.flows, notebook=True, show_nodes=False, euclidean=True)

# 3. First iteration 
skims = result.skim
new_od = np.zeros(4).reshape((2, 2))
new_od[0,1] = (A-skims)/B
print(new_od[0,1])
print(abs((new_od - old_od)))

# 4. Looping/iterating over STA's to determine the actual OD graph
i = 1 
max_iterations = 10
max_deviation_per_od = 5
while np.max(abs((new_od - old_od))) > max_deviation_per_od and i < max_iterations:
    # Perhaps MSA stap
    old_od = new_od
    old_graph =  od_graph_from_matrix(old_od, centroid_x, centroid_y)
    assignment = StaticAssignment(network,old_graph)
    result = assignment.run('dial_b')
    skims = result.skim
    new_od = np.zeros(4).reshape((2, 2))
    new_od[0,1] = (A-skims)/B
    i += 1
    print(abs((new_od - old_od)))
    

    Congratulations, you have now run an elastic assignment on a tolled toy network!

-----------------------------------------------------------------------------------------------------------------------

5. The last step is preparing for the Heeds iterations. The outer loop will decide on the best toll value, by ‘randomly’ trying toll values within a given range. To do this, the full network (containing roads, centroids and connectors) should preferably be saved to avoid reloading these files each iteration. In the elastic use-case, only the network remains static and is saved with a given path. The OD however will change here, exactly because of the elasticity matrix. It is not saved, but continuously updated each step of the process. What could be useful is saving the original OD so you do not lose this!


In [ ]:
HERE = os.path.dirname(os.path.realpath("__file__"))
HEEDS_data_path = HERE+ os.path.sep+os.pardir+os.path.sep+os.pardir+os.path.sep+'data_map' + os.path.sep + "HEEDS_input"
network_path = HEEDS_data_path + os.path.sep + 'network_with_centroids' + os.path.sep + "elastic_toy"
with open(network_path, 'wb') as network_file:
    dump(network, network_file)
    print(f'network saved at f{network_path}')

    Congratulations, you are now ready to go to the next step: HEEDS!